References

1.	http://excelsior-cjh.tistory.com/entry/5-Pandas를-이용한-Naver금융에서-주식데이터-가져오기
2.	https://financedata.github.io/posts/naver-finance-finstate-crawling.html
3.	https://gomjellie.github.io/파이썬/pandas/주식/2017/06/09/pandas-datareader-stock.html
4.	http://estenpark.tistory.com/353
5.	https://m.blog.naver.com/PostView.nhn?blogId=koo_m&logNo=221162347920&proxyReferer=https%3A%2F%2Fwww.google.com%2F
6.	http://blog.naver.com/PostView.nhn?blogId=suma_maple&logNo=220970555530
7.	http://dongsamb.com/web-scraping-using-python/
8.  https://github.com/datacamp/datacamp-community-tutorials
9.  https://www.datacamp.com/community/tutorials/finance-python-trading
10. https://www.scrapehero.com/scrape-yahoo-finance-stock-market-data/
11. https://www.facebook.com/financedata/
12. https://www.slideshare.net/plusjune/py-con-2014-47762297
13. https://medium.com/python-data/capm-analysis-calculating-stock-beta-as-a-regression-in-python-c82d189db536
14. https://www.quantopian.com/posts/computing-the-fama-french-factors-with-pipeline-1

In [1]:
import pandas as pd

In [2]:
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]
code_df.종목코드 = code_df['종목코드'].map('{:06d}'.format)
code_df = code_df[['회사명', '종목코드']]
code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'})

In [3]:
code_df.head()

,name,code
0,BYC,001460
1,CJ CGV,079160
2,CJ씨푸드,011150
3,CJ제일제당,097950
4,DB손해보험,005830


In [4]:
def get_url(item_name, code_df):
    code = code_df.query("name=='{}'".format(item_name))['code'].to_string(index=False)
    url = "http://finance.naver.com/item/sise_day.nhn?code={code}".format(code=code)
    
    print("요청 URL = {}".format(url))
    return url

item_name = 'BYC'
url = get_url(item_name, code_df)
#get precious data

요청 URL = http://finance.naver.com/item/sise_day.nhn?code=001460


In [5]:
df = pd.DataFrame()

In [6]:
for page in range(1, 10):
    pg_url = '{url}&page={page}'.format(url=url, page=page)
    df = df.append(pd.read_html(pg_url, header=0)[0], ignore_index=True)
#pages previously

In [7]:
df = df.dropna()

In [8]:
df.head()

,날짜,종가,전일비,시가,고가,저가,거래량
0,2018.02.06,301000,0,299000,301500,295000,818
1,2018.02.05,301000,1500,301000,304500,300500,652
2,2018.02.02,302500,2000,304500,304500,300500,437
3,2018.02.01,304500,0,307500,307500,303000,620
4,2018.01.31,304500,1500,303000,304500,302500,678


In [9]:
df = df.rename(columns={'날짜': 'date', '종가': 'close', '전일비': 'diff',
                       '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume'})

In [10]:
df[['close', 'diff', 'open', 'high', 'low', 'volume']] \
 = df[['close', 'diff', 'open', 'high', 'low', 'volume']].astype(int) # 1000 = 1 d

In [11]:
df['date'] = pd.to_datetime(df['date'])

In [12]:
df = df.sort_values(by=['date'], ascending=True)

In [13]:
df.head()

,date,close,diff,open,high,low,volume
89,2017-09-21,320500,5500,326000,326000,320500,704
88,2017-09-22,319000,1500,318500,319500,314500,786
87,2017-09-25,312500,6500,315000,330000,310000,569
86,2017-09-26,316000,3500,310000,323000,310000,206
85,2017-09-27,320000,4000,322000,326000,319500,337


In [14]:
#pip install plotly
import plotly.offline as offline
import plotly.graph_objs as go

offline.init_notebook_mode(connected=True)

trace = go.Scatter(x=df.date, y=df.close, name=item_name)

data = [trace]

layout = dict(
            title='{}의 종가(close) Time Series'.format(item_name),
            xaxis =dict(
                rangeselector=dict(
                buttons=list([
                    dict(count=1,
                        label='1m',
                        step='month',
                        stepmode='backward'),
                    dict(count=3,
                        label='3m',
                        step='month',
                        stepmode='backward'),
                    dict(count=6,
                        label='6m',
                        step='month',
                        stepmode='backward'),
                    dict(step='all')
                ])
            ),
            rangeslider=dict(),
            type='date'
        )
    )

fig = go.Figure(data=data, layout=layout)
offline.iplot(fig)